In [127]:
import re

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import *

In [140]:
 spark = SparkSession\
        .builder\
        .appName("TwitterSentimentAnalysis")\
        .master("local[*]")\
        .getOrCreate()

In [141]:
spark.sparkContext.setLogLevel('WARN')

In [142]:
lines = spark.readStream\
        .format("socket")\
        .option("host", "127.0.0.1")\
        .option("port", 3333)\
        .load()

22/11/18 01:02:43 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [143]:
tweet_df_string = lines.selectExpr("CAST(value AS STRING)")

tweets_tab = tweet_df_string.withColumn('word', explode(split(col('value'), ' '))) \
    .groupBy('word') \
    .count() \
    .sort('count', ascending=False). \
    filter(col('word').contains('#'))

22/11/18 01:02:45 WARN StateStore: Error managing HDFSStateStoreProvider[id = (op=0,part=165),dir = file:/private/var/folders/vw/8j0kbq6n5z7682q0r894_vxm0000gn/T/temporary-4fd2e3e1-97ce-4c0f-90cf-eb4fe9b3b57c/state/0/165], stopping management thread
22/11/18 01:02:45 WARN StateStore: Error running maintenance thread
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.sql.execution.streaming.state.StateStoreCoordinatorRef$.forExecutor(StateStoreCoordinator.scala:84)
	at org.apache.spark.sql.execution.streaming.state.StateStore$.coordinatorRef(StateStore.scala:653)
	at org.apache.spark.sql.execution.streamin

In [144]:
writeTweet = tweets_tab.writeStream. \
    outputMode("complete"). \
    format("memory"). \
    queryName("tweetquery"). \
    trigger(processingTime='2 seconds'). \
    start()

22/11/18 01:02:49 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/vw/8j0kbq6n5z7682q0r894_vxm0000gn/T/temporary-6d912829-b008-4f30-b826-a518fc45977c. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/11/18 01:02:49 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


22/11/18 01:02:58 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 2000 milliseconds, but spent 8822 milliseconds
22/11/18 01:03:20 WARN TextSocketMicroBatchStream: Stream closed by 127.0.0.1:3333


In [145]:
spark.stop()
